In [1]:
import time  
start = time.time()
prevstep = time.time()
def frmDate(now):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(now)) 

def logDiff(mesage):
    global prevstep
    end = time.time()
    diff = int(end - start)
    diffstep = int(end - prevstep)
    prevstep = end
    print(mesage,"--- diff-total:", diff, ",", \
          "diffstep:", diffstep, "," , \
          "start:", frmDate(start), "," , \
          "start_step:",  frmDate(prevstep), "," , \
          "end_step:", frmDate(end))
logDiff("init")    

init --- diff-total: 0 , diffstep: 0 , start: 2022-05-27 20:30:08 , start_step: 2022-05-27 20:30:08 , end_step: 2022-05-27 20:30:08


In [2]:
!pip3 install pandas h3 sqlalchemy boto3 oci delta-spark IPython

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
print("starting...")



# -*- coding: utf-8 -*-
"""part_1_mobility_script_generic(new_dataset)_new (2).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1dGNHBaJAS1w2hsZw4c_Kx4bFdQ0Kzdg_
"""

print("starting...")

from pyspark.sql.window import Window 
import pyspark
from pyspark import SparkContext

from pyspark.sql import functions as F
import pyspark.sql.functions as func


from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import col, window
import pyspark.sql.functions as func
from pyspark.sql.functions import desc
import datetime, time
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.window import Window
import pyspark.sql.functions as sf

import datetime as dt
import numpy as np
import os

from multiprocessing.pool import Pool
import pickle
import pandas as pd

from h3 import h3


from math import radians, cos, sin, asin, sqrt
from pyspark.sql.functions import lit
import json

from datetime import datetime, date, timedelta
 
import oci
from pyspark.sql import SparkSession
import sys
import boto3
from io import BytesIO

from functools import reduce
from pyspark.sql import DataFrame

from pyspark.sql.functions import round, col
from dateutil import tz
import sqlalchemy as db

import pyspark

from pyspark.sql import SparkSession
ociconf = oci.config.from_file()
from pyspark.sql.functions import broadcast
from IPython.display import display
import time

# new
from pyspark.sql import *
from pyspark import SparkConf, SparkContext 

# .config("spark.jars.packages", "io.delta:delta-core_2.12:1.1.0") \
# .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
# .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
# # config for 88 executor  -- change executor  count slaves 11=8c64g  11*8==88 cores, 11 * 64=704
spark = SparkSession.builder \
        .config("spark.driver.memory", "64g") \
        .config("spark.driver.cores", "8") \
        .config("spark.executor.memory", "64g") \
        .config("spark.executor.cores", "8") \
        .config("spark.scheduler.mode", "FAIR") \
        .config("spark.cores.max", "8") \
        .config("spark.delta.logStore.oci.impl","io.delta.storage.OracleCloudLogStore") \
        .getOrCreate() 
#.config("spark.executor.instances", "10")
spark.conf.set('fs.oci.client.auth.tenantId', "ocid1.tenancy.oc1..aaaaaaaamqxmfclvmrazpk4kt7ibkcfzfg2fvg4o2wyzemzu3n7tcpf6nvsa")
spark.conf.set('fs.oci.client.auth.userId', "ocid1.user.oc1..aaaaaaaa4gjumyz4kiowhzbyjzgajyo5bln565rntgboqod3t6lu4bpwl2ra")
spark.conf.set('fs.oci.client.auth.fingerprint', "ec:8f:0e:14:e0:31:cd:e4:d6:03:04:aa:aa:94:f9:a9")
spark.conf.set('fs.oci.client.auth.pemfilepath', "~/.oci/oci_api_key")
spark.conf.set('fs.oci.client.auth.tenantId.region', "us-ashburn-1")
spark.conf.set('fs.oci.client.hostname', "https://objectstorage.us-ashburn-1.oraclecloud.com")

# new sql 
# spark = SQLContext(spark.sparkContext)
## register data frame as a temporary table
# orders_df.registerTempTable("orders")

spark.sparkContext._conf.getAll()
logDiff("sparkContext") 

starting...
starting...


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/27 20:30:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/27 20:30:14 WARN FairSchedulableBuilder: Fair Scheduler configuration file not found so jobs will be scheduled in FIFO order. To use fair scheduling, configure pools in fairscheduler.xml or set spark.scheduler.allocation.file to a file that contains the configuration.


sparkContext --- diff-total: 8 , diffstep: 8 , start: 2022-05-27 20:30:08 , start_step: 2022-05-27 20:30:16 , end_step: 2022-05-27 20:30:16


In [4]:

data =[("James ","","Smith","36636","M",3000),
              ("Michael ","Rose","","40288","M",4000),
              ("Robert ","","Williams","42114","M",4000),
              ("Maria ","Anne","Jones","39192","F",4000),
              ("Jen","Mary","Brown","","F",-1)]
columns=["firstname","middlename","lastname","dob","gender","salary"]
df=spark.createDataFrame(data,columns)


In [5]:
start_time = time.time()
df.show(3)
logDiff("show df") 
print("--- %s seconds ---" % (time.time() - start_time))

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|  dob|gender|salary|
+---------+----------+--------+-----+------+------+
|   James |          |   Smith|36636|     M|  3000|
| Michael |      Rose|        |40288|     M|  4000|
|  Robert |          |Williams|42114|     M|  4000|
+---------+----------+--------+-----+------+------+
only showing top 3 rows

show df --- diff-total: 12 , diffstep: 4 , start: 2022-05-27 20:30:08 , start_step: 2022-05-27 20:30:20 , end_step: 2022-05-27 20:30:20
--- 2.5845351219177246 seconds ---


In [6]:
start_time = time.time()
delta_file="oci://mobility-data@bmmp5bv7olp2/ind/lifesight/silver_layer_city/test_delta1/"
df.write.format('delta').mode("overwrite") \
.save(delta_file)
print("--- %s seconds ---" % (time.time() - start_time))
logDiff("save df") 

22/05/27 20:30:22 WARN RestClient: Stream size to upload is 0 bytes, this could potentially lead to data corruption. If this is not intended, please make sure all the OCI SDK dependencies point to the same version
22/05/27 20:30:23 WARN RestClient: Stream size to upload is 0 bytes, this could potentially lead to data corruption. If this is not intended, please make sure all the OCI SDK dependencies point to the same version


--- 13.466702699661255 seconds ---
save df --- diff-total: 25 , diffstep: 13 , start: 2022-05-27 20:30:08 , start_step: 2022-05-27 20:30:33 , end_step: 2022-05-27 20:30:33


In [7]:
start_time = time.time()
parquet_file="oci://mobility-data@bmmp5bv7olp2/ind/lifesight/silver_layer_city/test_paruquet3/"
df.write.format('parquet').mode("overwrite") \
.save(parquet_file)
print("--- %s seconds ---" % (time.time() - start_time))
logDiff("save df") 

22/05/27 20:30:34 WARN RestClient: Stream size to upload is 0 bytes, this could potentially lead to data corruption. If this is not intended, please make sure all the OCI SDK dependencies point to the same version
22/05/27 20:30:34 WARN RestClient: Stream size to upload is 0 bytes, this could potentially lead to data corruption. If this is not intended, please make sure all the OCI SDK dependencies point to the same version
22/05/27 20:30:34 WARN RestClient: Stream size to upload is 0 bytes, this could potentially lead to data corruption. If this is not intended, please make sure all the OCI SDK dependencies point to the same version


--- 3.1383490562438965 seconds ---
save df --- diff-total: 28 , diffstep: 3 , start: 2022-05-27 20:30:08 , start_step: 2022-05-27 20:30:37 , end_step: 2022-05-27 20:30:37


22/05/27 20:30:36 WARN RestClient: Stream size to upload is 0 bytes, this could potentially lead to data corruption. If this is not intended, please make sure all the OCI SDK dependencies point to the same version


In [8]:
read_df = spark.read.format("delta").load(delta_file, header = 'true')
logDiff("load block_df") 
read_df.show(3)

load block_df --- diff-total: 29 , diffstep: 0 , start: 2022-05-27 20:30:08 , start_step: 2022-05-27 20:30:37 , end_step: 2022-05-27 20:30:37


+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|  dob|gender|salary|
+---------+----------+--------+-----+------+------+
|  Robert |          |Williams|42114|     M|  4000|
|   Maria |      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+
only showing top 3 rows



In [9]:
read_df = spark.read.format("parquet").load(parquet_file, header = 'true')
logDiff("load block_df") 
read_df.show(3)

load block_df --- diff-total: 31 , diffstep: 2 , start: 2022-05-27 20:30:08 , start_step: 2022-05-27 20:30:39 , end_step: 2022-05-27 20:30:39
+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|  dob|gender|salary|
+---------+----------+--------+-----+------+------+
|  Robert |          |Williams|42114|     M|  4000|
|   Maria |      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown|     |     F|    -1|
+---------+----------+--------+-----+------+------+
only showing top 3 rows

